## Загрузка данных

In [7]:
import pandas as pd
import numpy as np
import ydata_profiling

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import phik
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.pipeline import Pipeline

from datetime import datetime
from IPython.core.display import display, HTML

pd.set_option('display.max_columns', None)

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import warnings
warnings.filterwarnings(action='ignore')

display(HTML("<style>.container { width:80% !important; }</style>"))

In [8]:
path_apparel = r'C:\Users\user\OneDrive\Документы\GitHub\Pet-Projects\Предсказание покупки\Data\apparel-purchases.csv'
path_gifts = r'C:\Users\user\OneDrive\Документы\GitHub\Pet-Projects\Предсказание покупки\Data\gifts-purchases.csv'
path_marketplace = r'C:\Users\user\OneDrive\Документы\GitHub\Pet-Projects\Предсказание покупки\Data\marketplace-purchases.csv'

In [9]:
df_apparel = pd.read_csv(path_apparel)
df_gifts = pd.read_csv(path_gifts)
df_marketplace = pd.read_csv(path_marketplace)

Изучим данные каждого датасета:

### `df_apparel`

In [28]:
#ydata_profiling.ProfileReport(df_apparel, explorative=True).to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
def print_dataset_info(df, dataset_name):
    print(f"Информация о датасете {dataset_name}:")
    print(df.info())
    print("\nПервые 5 строк датасета:")
    display(df.head())
    print("\nКоличество пропусков:")
    display(df.isna().sum().sum())
    print("\nКоличество дубликатов:")
    display(df.duplicated().sum())

In [24]:
print_dataset_info(df_apparel, 'df_apparel')

Информация о датасете df_apparel:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133104 entries, 0 to 133103
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   client_id       133104 non-null  int64  
 1   quantity        133104 non-null  int64  
 2   price           133104 non-null  float64
 3   date            133104 non-null  object 
 4   message_id      133104 non-null  object 
 5   recommended_by  133104 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 6.1+ MB
None

Первые 5 строк датасета:


,client_id,quantity,price,date,message_id,recommended_by
0,1515915625468068833,1,3499.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
1,1515915625468068833,1,3499.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
2,1515915625468068833,1,3499.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
3,1515915625468068833,1,2450.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
4,1515915625468068833,1,2450.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message



Количество пропусков:


0


Количество дубликатов:


59250

Посмотрим количество уникальных сообщений у каждого клиента

In [27]:
client_message_counts = df_apparel.groupby('client_id')['message_id'].nunique()

client_message_counts[client_message_counts > 1]

client_id
1515915625468061170    3
1515915625468061994    2
1515915625468063775    2
1515915625468064438    4
1515915625468065092    2
                      ..
1515915625982037551    2
1515915625982466851    3
1515915625983450961    2
1515915625983782421    2
1515915625984308128    2
Name: message_id, Length: 2497, dtype: int64

In [30]:
df_apparel[df_apparel.duplicated()]

,client_id,quantity,price,date,message_id,recommended_by
1,1515915625468068833,1,3499.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
2,1515915625468068833,1,3499.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
4,1515915625468068833,1,2450.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
5,1515915625468068833,1,2450.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
10,1515915625468079941,1,1499.0,2023-09-01,1515915625468079941-13718-64e899a67a51d,bulk_message
...,...,...,...,...,...,...
133083,1515915625909099325,1,1499.0,2022-12-30,1515915625806806683-9110-63ae8d4faa490,bulk_message
133088,1515915625909141235,1,399.0,2022-12-30,1515915625817034221-9110-63ae8df9aae33,bulk_message
133090,1515915625909141235,1,399.0,2022-12-30,1515915625817034221-9110-63ae8df9aae33,bulk_message
133100,1515915625909375164,1,2399.0,2022-12-31,1515915625490956133-9110-63ae8ee3e5e7f,bulk_message


Одному клиенту могут быть присвоены разные идентификаторы сообщений

- Датасет содержит 133104 строки и 6 столбцов
- Пропуски отсутствуют
- Имеется 59250 дубликатов
- Признак `recommended_by` имеет только одно значение для всех строк

### `df_gifts`

In [95]:
ydata_profiling.ProfileReport(df_gifts, explorative=True).to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

- Датасет содержит 811 строк и 6 столбцов
- Пропуски отсутствуют
- Имеется ~4% дубликатов
- Признак `recommended_by`имеет только одно значение для всех строк

### `df_marketplace`

In [96]:
ydata_profiling.ProfileReport(df_marketplace, explorative=True).to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

- Датасет содержит 48182 строк и 6 столбцов
- Пропуски отсутствуют
- Имеется ~3% дубликатов
- Имеется два временных признака

Структура у датасетов очень похожая, поэтому принято решение объединить их в один датасет

### Объединенный датасет

Объединим датасеты и добавим столбец `source` для понимания источника данных

In [97]:
df_apparel['source'] = 'apparel'
df_gifts['source'] = 'gifts'
df_marketplace['source'] = 'marketplace'

# Объединяем данные по строкам
df = pd.concat([df_apparel, df_gifts, df_marketplace])

In [98]:
ydata_profiling.ProfileReport(df, explorative=True).to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\user\anaconda3\lib\site-packages\ydata_profiling\model\pandas\correlations_pandas.py:84: FutureWarning:

reindexing with a non-unique Index is deprecated and will raise in a future version.

C:\Users\user\anaconda3\lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning:

There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'cannot reindex on an axis with duplicate labels')



Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Признак `client_id` имеет числовой тип, хотя он обозначает конкретного клиента. Преобразуем его

In [99]:
df['client_id'] = df['client_id'].astype(str)

Дата в столбце `date`имеет разный формат. Приведем к единому формату

In [100]:
df['date'] = pd.to_datetime(df['date']).dt.date

Посмотрим, могут ли исходные датасеты иметь одинаковые `client_id`

In [101]:
client_source_counts = df.groupby('client_id')['source'].nunique()

clients_with_multiple_source = client_source_counts[client_source_counts > 1]

clients_with_multiple_source

Series([], Name: source, dtype: int64)

Пересечений client_id между датасетами нет.

Имеются признаки, которые навряд ли помогут нам в предсказании будущей покупки. По моему мнению, таковыми являются:
- `message_id` - это идентификатор сообщения, который навряд ли связан с историей покупок клиента
- `recommended_by` - в каждой строке одинаковые значения
- `created_at` - дублирует признак `date`

In [102]:
df.drop(['message_id', 'recommended_by', 'created_at'], axis=1, inplace=True)

In [103]:
df.head()

,client_id,quantity,price,date,source
0,1515915625468068833,1,3499.0,2023-09-08,apparel
1,1515915625468068833,1,3499.0,2023-09-08,apparel
2,1515915625468068833,1,3499.0,2023-09-08,apparel
3,1515915625468068833,1,2450.0,2023-09-08,apparel
4,1515915625468068833,1,2450.0,2023-09-08,apparel


## Фиче-инженеринг

Узнаем суммарное значение покупки каждой позиции

In [104]:
df['total_price'] = df['quantity'] * df['price']

Упорядочим датасет по `client_id` и `date`

In [105]:
df = df.sort_values(['client_id', 'date'],  ascending=[True, False])

Определим крайнюю дату датасета

In [106]:
last_date = df['date'].max()

last_date

datetime.date(2023, 10, 26)

Определим дату начала среза

In [107]:
start_date = last_date - pd.Timedelta(days=30)

start_date

datetime.date(2023, 9, 26)

Отберем строки, которые попадают в срез

In [108]:
last_30_days = df[df['date'] >= start_date]

last_30_days.head()

,client_id,quantity,price,date,source,total_price
1827,1515915625440981562,1,25798.0,2023-10-01,marketplace,25798.0
1828,1515915625440995965,1,14998.0,2023-10-11,marketplace,14998.0
863,1515915625440996406,1,6118.0,2023-09-28,marketplace,6118.0
1829,1515915625441001906,1,2.0,2023-10-03,marketplace,2.0
866,1515915625441007350,1,7998.0,2023-09-29,marketplace,7998.0


Составим список клиентов, которые совершали покупку в последние 30 дней

In [109]:
client_list = last_30_days['client_id'].unique().tolist()

client_list[:5]

['1515915625440981562',
 '1515915625440995965',
 '1515915625440996406',
 '1515915625441001906',
 '1515915625441007350']

Сгруппируем датасет **df** по признаку `client_id`, чтобы узнать кто сколько всего купил

In [110]:
# сгруппируем данные по столбцу client_id и посчитаем сумму quantity и сумму price
df_client = df.groupby('client_id').agg({'quantity': 'sum', 'price': 'sum'}).reset_index()

df_client.head()

,client_id,quantity,price
0,1515915625440099873,2,33216.0
1,1515915625440939402,1,75998.0
2,1515915625440944408,3,81178.0
3,1515915625440947454,4,4486.0
4,1515915625440952940,1,479.0


In [111]:
df_client.shape

(47004, 3)

Вновь создадим признак `source`

In [112]:
# Удаляем дубликаты из df
df_unique = df.drop_duplicates()

# Группируем df_unique по client_id и source и выбираем первое значение
df_unique_grouped = df_unique.groupby(['client_id', 'source']).first().reset_index()

# Объединяем df_client с df_unique_grouped по client_id, оставив только столбцы client_id и source из df_unique_grouped
df_client = pd.merge(df_client, df_unique_grouped[['client_id', 'source']], on='client_id', how='left')

In [113]:
# Выводим результат
df_client

,client_id,quantity,price,source
0,1515915625440099873,2,33216.0,marketplace
1,1515915625440939402,1,75998.0,marketplace
2,1515915625440944408,3,81178.0,marketplace
3,1515915625440947454,4,4486.0,marketplace
4,1515915625440952940,1,479.0,marketplace
...,...,...,...,...
46999,1515915626005014483,1,7534.0,gifts
47000,1515915626010039507,2,26039.0,gifts
47001,1515915626011484905,5,32913.0,gifts
47002,1515915626012131867,1,25834.0,gifts


### Минимальное количество дней между покупками

Определим минимальное количество дней между двумя последовательными транзакциями пользователя

In [114]:
df['date_diff'] = df.groupby('client_id')['date'].diff()

In [115]:
df.head(10)

,client_id,quantity,price,date,source,total_price,date_diff
25171,1515915625440099873,1,1218.0,2023-08-09,marketplace,1218.0,NaT
2644,1515915625440099873,1,31998.0,2023-07-16,marketplace,31998.0,-24 days
2396,1515915625440939402,1,75998.0,2023-01-30,marketplace,75998.0,NaT
24292,1515915625440944408,1,73998.0,2023-06-21,marketplace,73998.0,NaT
0,1515915625440944408,1,6298.0,2023-05-02,marketplace,6298.0,-50 days
434,1515915625440944408,1,882.0,2023-03-01,marketplace,882.0,-62 days
16775,1515915625440947454,1,286.0,2022-12-24,marketplace,286.0,NaT
16776,1515915625440947454,1,3998.0,2022-12-24,marketplace,3998.0,0 days
4333,1515915625440947454,1,2.0,2022-11-06,marketplace,2.0,-48 days
4334,1515915625440947454,1,200.0,2022-11-06,marketplace,200.0,0 days


Отбрасываем строки, где `date_diff` является *NaT* (означает, что это первая транзакция для клиента)

In [116]:
df = df[~df['date_diff'].isna()]

In [117]:
# преобразуем date_diff в дни
df['date_diff'] = df['date_diff'].dt.days

In [118]:
# переведем в положительные значения разницу дней между покупками
df['date_diff'] = df['date_diff'].abs()

In [119]:
# удалим значения 0, т.к. это разница между разными покупками в один и тот же день
df = df[df['date_diff'] > 0]

In [120]:
df.head()

,client_id,quantity,price,date,source,total_price,date_diff
2644,1515915625440099873,1,31998.0,2023-07-16,marketplace,31998.0,24
0,1515915625440944408,1,6298.0,2023-05-02,marketplace,6298.0,50
434,1515915625440944408,1,882.0,2023-03-01,marketplace,882.0,62
4333,1515915625440947454,1,2.0,2022-11-06,marketplace,2.0,48
4336,1515915625440966943,1,478.0,2022-11-26,marketplace,478.0,27


Создадим список с минимальным количеством дней между покупками

In [121]:
min_days_between_transactions = df.groupby('client_id')['date_diff'].min().reset_index()

In [122]:
min_days_between_transactions.head()

,client_id,date_diff
0,1515915625440099873,24
1,1515915625440944408,50
2,1515915625440947454,48
3,1515915625440966943,27
4,1515915625440978763,110


In [123]:
df_client = df_client.merge(min_days_between_transactions, on='client_id', how='left')

In [124]:
df_client.rename(columns={'date_diff': 'min_date_diff'}, inplace=True)

### Максимальное количество дней между покупками

In [125]:
max_days_between_transactions = df.groupby('client_id')['date_diff'].max().reset_index()

In [126]:
max_days_between_transactions.head()

,client_id,date_diff
0,1515915625440099873,24
1,1515915625440944408,62
2,1515915625440947454,48
3,1515915625440966943,27
4,1515915625440978763,110


In [127]:
df_client = df_client.merge(max_days_between_transactions, on='client_id', how='left')

In [128]:
df_client.rename(columns={'date_diff': 'max_date_diff'}, inplace=True)

In [129]:
df_client.head()

,client_id,quantity,price,source,min_date_diff,max_date_diff
0,1515915625440099873,2,33216.0,marketplace,24.0,24.0
1,1515915625440939402,1,75998.0,marketplace,NaN,NaN
2,1515915625440944408,3,81178.0,marketplace,50.0,62.0
3,1515915625440947454,4,4486.0,marketplace,48.0,48.0
4,1515915625440952940,1,479.0,marketplace,NaN,NaN


### Среднее количество дней между покупками

In [130]:
mean_days_between_transactions = df.groupby('client_id')['date_diff'].agg(['sum', 'count']).reset_index()

In [131]:
mean_days_between_transactions['mean'] =  mean_days_between_transactions['sum'] / mean_days_between_transactions['count']

In [132]:
df_client = df_client.merge(mean_days_between_transactions[['client_id', 'mean']], on='client_id', how='left')

In [133]:
df_client.rename(columns={'mean': 'mean_date_diff'}, inplace=True)

In [134]:
df_client

,client_id,quantity,price,source,min_date_diff,max_date_diff,mean_date_diff
0,1515915625440099873,2,33216.0,marketplace,24.0,24.0,24.0
1,1515915625440939402,1,75998.0,marketplace,NaN,NaN,NaN
2,1515915625440944408,3,81178.0,marketplace,50.0,62.0,56.0
3,1515915625440947454,4,4486.0,marketplace,48.0,48.0,48.0
4,1515915625440952940,1,479.0,marketplace,NaN,NaN,NaN
...,...,...,...,...,...,...,...
46999,1515915626005014483,1,7534.0,gifts,NaN,NaN,NaN
47000,1515915626010039507,2,26039.0,gifts,NaN,NaN,NaN
47001,1515915626011484905,5,32913.0,gifts,NaN,NaN,NaN
47002,1515915626012131867,1,25834.0,gifts,NaN,NaN,NaN


Удалим строки с отсутствующими значениями в столбцах `min_date_diff`, `max_date_diff` и `mean_date_diff`

In [135]:
df_client = df_client.dropna()

In [136]:
df_client

,client_id,quantity,price,source,min_date_diff,max_date_diff,mean_date_diff
0,1515915625440099873,2,33216.0,marketplace,24.0,24.0,24.0
2,1515915625440944408,3,81178.0,marketplace,50.0,62.0,56.0
3,1515915625440947454,4,4486.0,marketplace,48.0,48.0,48.0
10,1515915625440966943,2,18476.0,marketplace,27.0,27.0,27.0
13,1515915625440978763,5,134490.0,marketplace,110.0,110.0,110.0
...,...,...,...,...,...,...,...
46780,1515915625984308128,3,4597.0,apparel,43.0,43.0,43.0
46795,1515915625984937793,4,10596.0,apparel,1.0,1.0,1.0
46802,1515915625985115331,3,17266.0,gifts,10.0,10.0,10.0
46940,1515915625994223181,6,8609.0,apparel,1.0,1.0,1.0


### RFM анализ

#### Recency

In [137]:
#Recency = df.groupby('client_id')['date'].max().reset_index()

In [138]:
#Recency = last_date - Recency['date']

In [139]:
recency = (last_date - df.groupby("client_id").agg({"date":"max"})).reset_index()

In [140]:
recency['date'] = recency['date'].apply(lambda x: x.days)

In [141]:
recency.head()

,client_id,date
0,1515915625440099873,102
1,1515915625440944408,177
2,1515915625440947454,354
3,1515915625440966943,334
4,1515915625440978763,307


In [142]:
df_client = df_client.merge(recency[['client_id', 'date']], on='client_id', how='left')

In [143]:
df_client.rename(columns={'date': 'recency'}, inplace=True)

In [144]:
df_client.head()

,client_id,quantity,price,source,min_date_diff,max_date_diff,mean_date_diff,recency
0,1515915625440099873,2,33216.0,marketplace,24.0,24.0,24.0,102
1,1515915625440944408,3,81178.0,marketplace,50.0,62.0,56.0,177
2,1515915625440947454,4,4486.0,marketplace,48.0,48.0,48.0,354
3,1515915625440966943,2,18476.0,marketplace,27.0,27.0,27.0,334
4,1515915625440978763,5,134490.0,marketplace,110.0,110.0,110.0,307


#### Frequency

In [145]:
frequency = df.groupby("client_id").agg({"date":"nunique"}).reset_index()

In [146]:
df_client = df_client.merge(frequency[['client_id', 'date']], on='client_id', how='left')

In [147]:
df_client.rename(columns={'date': 'frequency'}, inplace=True)

In [148]:
df_client.head()

,client_id,quantity,price,source,min_date_diff,max_date_diff,mean_date_diff,recency,frequency
0,1515915625440099873,2,33216.0,marketplace,24.0,24.0,24.0,102,1
1,1515915625440944408,3,81178.0,marketplace,50.0,62.0,56.0,177,2
2,1515915625440947454,4,4486.0,marketplace,48.0,48.0,48.0,354,1
3,1515915625440966943,2,18476.0,marketplace,27.0,27.0,27.0,334,1
4,1515915625440978763,5,134490.0,marketplace,110.0,110.0,110.0,307,1


#### Monetary

In [149]:
df_client['monetary'] = df_client['price']
df_client.drop('price', axis=1, inplace=True)

In [150]:
df_client.head()

,client_id,quantity,source,min_date_diff,max_date_diff,mean_date_diff,recency,frequency,monetary
0,1515915625440099873,2,marketplace,24.0,24.0,24.0,102,1,33216.0
1,1515915625440944408,3,marketplace,50.0,62.0,56.0,177,2,81178.0
2,1515915625440947454,4,marketplace,48.0,48.0,48.0,354,1,4486.0
3,1515915625440966943,2,marketplace,27.0,27.0,27.0,334,1,18476.0
4,1515915625440978763,5,marketplace,110.0,110.0,110.0,307,1,134490.0


#### Кластеризация данных с использованием алгоритма k-средних

In [151]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_client[['recency', 'frequency', 'monetary']])

In [152]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42) 
kmeans.fit(scaled_data)
labels = kmeans.labels_

C:\Users\user\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [153]:
df_client['cluster'] = labels

In [154]:
df_client.head()

,client_id,quantity,source,min_date_diff,max_date_diff,mean_date_diff,recency,frequency,monetary,cluster
0,1515915625440099873,2,marketplace,24.0,24.0,24.0,102,1,33216.0,1
1,1515915625440944408,3,marketplace,50.0,62.0,56.0,177,2,81178.0,1
2,1515915625440947454,4,marketplace,48.0,48.0,48.0,354,1,4486.0,0
3,1515915625440966943,2,marketplace,27.0,27.0,27.0,334,1,18476.0,0
4,1515915625440978763,5,marketplace,110.0,110.0,110.0,307,1,134490.0,0


#### Обозначение таргета

Поставим по умолчанию значение 0

In [156]:
df_client['target'] = 0

In [157]:
# Устанавливаем значение 1 для клиентов, которые есть в списке client_list
df_client.loc[df_client['client_id'].isin(client_list), 'target'] = 1

In [158]:
df_client.head()

,client_id,quantity,source,min_date_diff,max_date_diff,mean_date_diff,recency,frequency,monetary,cluster,target
0,1515915625440099873,2,marketplace,24.0,24.0,24.0,102,1,33216.0,1,0
1,1515915625440944408,3,marketplace,50.0,62.0,56.0,177,2,81178.0,1,0
2,1515915625440947454,4,marketplace,48.0,48.0,48.0,354,1,4486.0,0,0
3,1515915625440966943,2,marketplace,27.0,27.0,27.0,334,1,18476.0,0,0
4,1515915625440978763,5,marketplace,110.0,110.0,110.0,307,1,134490.0,0,0


## Разделение на выборки

Преобразуем признак source  в числовой формат

In [219]:
le = LabelEncoder()

In [220]:
df_client['source_encoded'] = le.fit_transform(df_client['source'])

In [221]:
features = df_client.drop(['target', 'source'], axis=1)  # Исключаем исходный категориальный признак
target = df_client['target']

In [222]:
target.value_counts()

0    5702
1     798
Name: target, dtype: int64

Целевой признак несбалансирован

Используем SMOTE для балансировки

In [223]:
from imblearn.over_sampling import SMOTE

In [224]:
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.25,
    stratify=target,
    random_state=42)

In [225]:
smote = SMOTE(random_state=42)

In [226]:
# Применяем SMOTE
features_SMOTE, target_SMOTE = smote.fit_resample(features_train, target_train)

## Обучение модели

In [227]:
from sklearn.model_selection import cross_val_score

In [233]:
# Инициализируем CatBoostClassifier
model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=False)

In [238]:
cv_result = cross_val_score(model, features_SMOTE, target_SMOTE, cv = 5 , scoring = "recall")

In [239]:
cv_result

array([0.96608187, 0.99299065, 0.99532164, 0.99415205, 0.99415205])

In [237]:
# Определяем модель CatBoost
model = CatBoostClassifier(loss_function="Logloss", verbose=False)

# Определяем сетку параметров для поиска
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [6, 8, 10]
}

# Создаем объект GridSearchCV
grid_catboost = GridSearchCV(estimator=model,
                             param_grid=param_grid,
                             scoring='accuracy',
                             cv=5,
                             n_jobs=-1)

# Обучаем модель с помощью кросс-валидации и поиска по сетке параметров
grid_catboost.fit(features_SMOTE, target_SMOTE)

# Получаем лучшие параметры
best_params = grid_catboost.best_params_
print("Лучшие параметры:", best_params)

# Получаем лучшую модель
best_model = grid_catboost.best_estimator_

# Оцениваем модель на тестовых данных
cv_result = cross_val_score(best_model, features_test, target_test, cv=5, scoring="accuracy")
print("Результаты кросс-валидации:", cv_result)

Лучшие параметры: {'depth': 10, 'iterations': 300, 'learning_rate': 0.2}
Результаты кросс-валидации: [0.93538462 0.95384615 0.96615385 0.95076923 0.94461538]


Узнаем метрики accuracy, precision, recall данной модели

In [240]:
predicted_labels = grid_catboost.predict(features_test)

In [241]:
accuracy_catboost = accuracy_score(target_test, predicted_labels)
precision_catboost = precision_score(target_test, predicted_labels)
recall_catboost = recall_score(target_test, predicted_labels)

In [242]:
print(accuracy_catboost)
print(precision_catboost)
print(recall_catboost)

0.955076923076923
0.7603305785123967
0.9246231155778895


## Важность признаков

In [245]:
feature_importances = best_model.get_feature_importance()

In [248]:
feature_importances_df = pd.DataFrame({'feature': features.columns, 'importance': feature_importances})


In [250]:
feature_importances_df = feature_importances_df.sort_values(by='importance', ascending=False)

In [251]:
feature_importances_df

,feature,importance
5,recency,45.252172
3,max_date_diff,20.543593
4,mean_date_diff,9.360346
2,min_date_diff,7.817993
7,monetary,4.743666
8,cluster,3.445116
6,frequency,3.096208
1,quantity,3.002954
9,source_encoded,2.737952
0,client_id,0.000000


Самый важный признак: 'recency'

## Прогнозирование повторных покупок

Найдем пользователей из нашего датасета df_clients, которые, вероятней всего, сделают повторную покупку в течении ближайших 30 дней

In [253]:
features_to_use = [col for col in df_client.columns if col != 'target']

In [255]:
predictions = best_model.predict(df_client[features_to_use])

In [256]:
df_client['future_purchase_prediction'] = predictions

In [260]:
potential_buyers = df_client[df_client['future_purchase_prediction'] == 1]

In [261]:
print(potential_buyers[['client_id', 'future_purchase_prediction']])

                client_id  future_purchase_prediction
5     1515915625440988920                           1
7     1515915625440995965                           1
8     1515915625440996406                           1
9     1515915625441007350                           1
16    1515915625441084418                           1
...                   ...                         ...
6435  1515915625952926667                           1
6439  1515915625956783429                           1
6476  1515915625977606982                           1
6498  1515915625994223181                           1
6499  1515915625997391318                           1

[841 rows x 2 columns]
